# Libraries Import

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import random


# Reading data

In [2]:
train_data = open(".//Data//protein-secondary-structure.train", "r")
test_data = open(".//Data//protein-secondary-structure.test", "r")

# Selecting the windows size and (training, validation) spliting

In [3]:
window_slide = 5
train_size = 80
validation_size = 20

# Parsing the train data

In [4]:
d_train = train_data.readline()
line = 1
x_train = []
y_train = []
nb_proteine = 0
while d_train:
    if line > 8:
        if len(d_train) == 1:
            pass
        elif d_train == '<>\n':
            if len(x_train) !=0 and x_train[len(x_train) -1] != '%':
                x_train.append('%')
            nb_proteine += 1
            x_train.append('%')
        elif d_train == 'end\n':
            x_train.append('%')
        elif d_train == '<end>\n':
            x_train.append('%')
        else:
            x_train.append(d_train[0])
            y_train.append(d_train[2])
            
        
    d_train = train_data.readline()
    line += 1

# Parsing the test data

In [5]:

line = 1
x_test = []
y_test = []
for k in range(int(window_slide/2) - 1):
    x_test.append('%')
    
d_test = test_data.readline()
while d_test:
    if line > 8:
        if len(d_test) == 1:
            pass
        elif d_test == '<>\n':
            if len(x_test)!=0 and x_test[len(x_test) -1] != '%':
                x_test.append('%')
            x_test.append('%')
        elif d_test == '<end>\n':
            x_test.append('%')
            for k in range(int(window_slide/2) - 1):
                x_test.append('%')
        else:
            x_test.append(d_test[0])
            y_test.append(d_test[2])
        
    d_test = test_data.readline()
    line += 1

# Split train data into Training, Validation data

In [6]:
nb_train = int(nb_proteine * train_size)
nb_valid = nb_proteine - nb_train
x_train_split = {'x_train_':[],'x_valid_':[]}
y_train_split = {'y_train_':[],'y_valid_':[]}
length_train_ = 0
length_valid_ = 0

In [7]:
l = 0
r = 0
for k in range(int(window_slide/2) - 1):
                x_train_split['x_train_'].append('%')
for k in range(int(window_slide/2) - 1):
                x_train_split['x_valid_'].append('%')
        
for k in range(len(x_train)):
    if x_train[k] == '%' and (k + 1)!= len(x_train) and x_train[k + 1] != '%':
        r = random.choice([0,1])
        if r ==0 and len(y_train_split['y_train_']) == nb_train:
            r = 1
        if r ==1 and len(y_train_split['y_valid_']) == nb_valid:
            r = 0
        if r == 0:
            length_train_ += 1
        else:
            length_valid_ += 1
            
    if r == 0:
        x_train_split['x_train_'].append(x_train[k])
        if x_train[k] != '%':
            y_train_split['y_train_'].append(y_train[l])
            l += 1
    else:
        x_train_split['x_valid_'].append(x_train[k])
        if x_train[k] != '%':
            y_train_split['y_valid_'].append(y_train[l])
            l += 1
            
for k in range(int(window_slide/2) - 1):
                x_train_split['x_train_'].append('%')
for k in range(int(window_slide/2) - 1):
                x_train_split['x_valid_'].append('%')
    

In [8]:
#print(len(y_train_split['y_valid_']) + len(y_train_split['y_train_']))
#print(len(y_train))
#print(length_train_+length_valid_)
#print(x_train_split['x_train_'])

In [9]:
#print(x_train[int(window_slide/2):21 + int(window_slide/2)])
#print(y_train[:21])

In [10]:
#print(x_test[int(window_slide/2):21 + int(window_slide/2)])
#print(y_test[:21])

# Coversion tables

In [11]:
convesion_input_data = {'%':0,'A':1,'C':2, 'D':3,'E':4,'F':5,'G':6, 'H':7, 'I':8, 'K':9, 'L':10, 'M':11, 'N':12, 'P':13,
                    'Q':14, 'R':15, 'S':16, 'T':17, 'V':18, 'W':19, 'Y':20}
convesion_output_data = {'_':0, 'e':1, 'h':2}

# Conversion to numbers

In [12]:
x_train_converted = []
y_train_converted = []
x_valid_converted = []
y_valid_converted = []
x_test_converted = []
y_test_converted = []

In [13]:
for k in range(len(x_train_split['x_train_'])):
    x_train_converted.append(convesion_input_data[x_train_split['x_train_'][k]])
    
for k in range(len(x_train_split['x_valid_'])):
    x_valid_converted.append(convesion_input_data[x_train_split['x_valid_'][k]])
    
for k in range(len(x_test)):
    x_test_converted.append(convesion_input_data[x_test[k]])
    
for k in range(len(y_train_split['y_train_'])):
    y_train_converted.append(convesion_output_data[y_train_split['y_train_'][k]])
    
for k in range(len(y_train_split['y_valid_'])):
    y_valid_converted.append(convesion_output_data[y_train_split['y_valid_'][k]])
    
for k in range(len(y_test)):
    y_test_converted.append(convesion_output_data[y_test[k]])

# 1-Hot encoding

In [14]:
x_train_one_hot = np.array(F.one_hot(torch.tensor(x_train_converted), num_classes=21))
x_valid_one_hot = np.array(F.one_hot(torch.tensor(x_valid_converted),  num_classes=21))
x_test_one_hot = np.array(F.one_hot(torch.tensor(x_test_converted),  num_classes=21))

y_train_one_hot = np.array(F.one_hot(torch.tensor(y_train_converted),  num_classes=3))
y_valid_one_hot = np.array(F.one_hot(torch.tensor(y_valid_converted), num_classes=3))
y_test_one_hot = np.array(F.one_hot(torch.tensor(y_test_converted), num_classes=3))

In [15]:
print(x_train_one_hot[0])

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [16]:
#print(y_test_converted[:21])

In [17]:
#print(y_test_one_hot[:21][:])

# Concatenation for training

In [18]:
x_train_final = []
x_valid_final = []
x_test_final = []

In [19]:
for k in range(len(x_train_converted)):
    if x_train_converted[k] != 0:
        data = []
        val = int(window_slide/2)
        for l in range(window_slide):
            data.extend(x_train_one_hot[l + k - val].copy())
        x_train_final.append(data.copy())
        
for k in range(len(x_valid_converted)):
    if x_valid_converted[k] != 0:
        data = []
        val = int(window_slide/2)
        for l in range(window_slide):
            data.extend(x_valid_one_hot[l + k - val].copy())
        x_valid_final.append(data.copy())
        
for k in range(len(x_test_converted)):
    if x_test_converted[k] != 0:
        data = []
        val = int(window_slide/2)
        for l in range(window_slide):
            data.extend(x_test_one_hot[l + k - val].copy())
        x_test_final.append(data.copy())

In [20]:
x_train_final = np.array(x_train_final,dtype=int)
x_valid_final = np.array(x_valid_final,dtype=int)
x_test_final = np.array(x_test_final,dtype=int)
#print(x_train_final[0])
#print(y_train_one_hot[0])

# Model

In [21]:
class MLP(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(MLP, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 3)
            self.softmax = torch.nn.Softmax(dim=1)
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.softmax(output)
            return output

In [22]:
model = MLP(105, 100)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

# Training

In [24]:
model.train()
epoch = 20
x_train_final = torch.tensor(x_train_final,  dtype=torch.float)
for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_train_final)
    # Compute Loss
    loss = criterion(y_pred, torch.tensor(y_train_one_hot,  dtype=torch.float))
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.6475067734718323
Epoch 1: train loss: 0.6471641659736633
Epoch 2: train loss: 0.6468229293823242
Epoch 3: train loss: 0.6464830636978149
Epoch 4: train loss: 0.6461443305015564
Epoch 5: train loss: 0.6458069682121277
Epoch 6: train loss: 0.6454707980155945
Epoch 7: train loss: 0.6451359391212463
Epoch 8: train loss: 0.6448025107383728
Epoch 9: train loss: 0.64447021484375
Epoch 10: train loss: 0.6441392302513123
Epoch 11: train loss: 0.6438096165657043
Epoch 12: train loss: 0.6434812545776367
Epoch 13: train loss: 0.6431540846824646
Epoch 14: train loss: 0.6428281664848328
Epoch 15: train loss: 0.6425036191940308
Epoch 16: train loss: 0.642180323600769
Epoch 17: train loss: 0.6418582797050476
Epoch 18: train loss: 0.6415374875068665
Epoch 19: train loss: 0.6412180066108704


# Test

In [27]:
model.eval()
y_pred = model(torch.tensor(x_test_final,  dtype=torch.float))
loss = criterion(y_pred, torch.tensor(y_test_one_hot,  dtype=torch.float))
print('Test loss' , loss.item())

Test loss 0.6414985656738281
